In [1]:
include("/home/kilian/wrk/assbiass/src/obs_funcs.jl")
include("/home/kilian/wrk/assbiass/src/mod_funcs.jl")

INFO: Checking out PyCall master...
INFO: Pulling PyCall latest master...
INFO: No packages to install, update or remove
INFO: Building Conda
INFO: Building PyCall
Info: PyCall is using /home/kilian/.envs/astro3/bin/python (Python 3.6.6) at /home/kilian/.envs/astro3/bin/python, libpython = /usr/lib/libpython3.6m
Info: /home/kilian/.playground/share/assbiass/packages/v0.6/PyCall/deps/deps.jl has not changed
Info: /home/kilian/.playground/share/assbiass/packages/v0.6/PyCall/deps/PYTHON has been updated


massbincounts

In [2]:
# here are the HOD parameters to be used for each of the samples

# for the -18 magnitude threshold, parameters are taken from Table 3 in Zehavi et al. 2011
m_th_18 = -18.0
paramdict_18 = Dict("alpha" => 1.04,
                    "logM0" =>  9.81,
                    "logM1" => 12.42,
                    "logMmin" => 11.18,
                    "sigma_logM" => 0.19)

# for the -19 magnitude threshold, parameters are taken from Sinha et al. 2016 best-fit values
# from table 3, using the values for Planck Cosmology with Mvir (as Chinchilla halos are defined) and no PCA uncertainties
m_th_19 = -19.0
paramdict_19 = Dict("alpha" => 0.95,       #  1.02 in Zehavi    1.03 in Zentner
                    "logM0" => 11.14,      #  9.77
                    "logM1" => 12.80,      # 12.63             12.78 whether I use Zehavi M0 or not
                    "logMmin" => 11.58,    # 11.45             11.72
                    "sigma_logM" => 0.05)  #  0.19              0.70

# for the -20 magnitude threshold, parameters are taken from Table 2 of Zentner et al. 2016, using M_0 from Zehavi
m_th_20 = -20.0
paramdict_20 = Dict("alpha" => 1.16,       #  1.0 in Zehavi
                    "logM0" =>  12.35,
                    "logM1" => 13.23,      # 12.98
                    "logMmin" => 11.96,    # 11.83
                    "sigma_logM" => 0.38)  #  0.25

m_th_arr = [m_th_18, m_th_19, m_th_20]
paramdicts = [paramdict_18, paramdict_19, paramdict_20]

3-element Array{Dict{String,Float64},1}:
 Dict("sigma_logM"=>0.19,"logMmin"=>11.18,"alpha"=>1.04,"logM1"=>12.42,"logM0"=>9.81) 
 Dict("sigma_logM"=>0.05,"logMmin"=>11.58,"alpha"=>0.95,"logM1"=>12.8,"logM0"=>11.14) 
 Dict("sigma_logM"=>0.38,"logMmin"=>11.96,"alpha"=>1.16,"logM1"=>13.23,"logM0"=>12.35)

In [3]:
# set which sample to generate a mock for and grab the data
sample_num = 1
datdf, vols = load_groupdat(sample_num, quenform = "both")

(12208×46 DataFrames.DataFrame. Omitted printing of 39 columns
│ Row   │ foo    │ galid  │ groupid │ cenid  │ rband    │ Psat │ Mh         │
├───────┼────────┼────────┼─────────┼────────┼──────────┼──────┼────────────┤
│ 1     │ PROB10 │ 17     │ 2485    │ 17     │ -20.4593 │ 0.0  │ 1.5053e12  │
│ 2     │ PROB10 │ 22     │ 13924   │ 22     │ -19.4819 │ 0.0  │ 2.20027e11 │
│ 3     │ PROB10 │ 83     │ 824     │ 83     │ -20.8211 │ 0.0  │ 4.7694e12  │
│ 4     │ PROB10 │ 93     │ 607     │ 93     │ -21.3781 │ 0.0  │ 6.43239e12 │
│ 5     │ PROB10 │ 99     │ 11864   │ 99     │ -18.7129 │ 0.0  │ 2.64106e11 │
│ 6     │ PROB10 │ 164    │ 10733   │ 164    │ -20.0047 │ 0.0  │ 2.95793e11 │
│ 7     │ PROB10 │ 181    │ 6630    │ 181    │ -19.6391 │ 0.0  │ 5.08768e11 │
│ 8     │ PROB10 │ 195    │ 12160   │ 195    │ -18.9404 │ 0.0  │ 2.56719e11 │
│ 9     │ PROB10 │ 205    │ 7018    │ 205    │ -19.8112 │ 0.0  │ 4.77532e11 │
│ 10    │ PROB10 │ 209    │ 8375    │ 209    │ -19.7289 │ 0.0  │ 3.91487e11 │
│

In [4]:
# get another dataframe of just the quenched galaxies and their masses for counting quenched fraction
quendf = datdf[datdf[:Dn4000] .> 1.6, [:log10M]]

,log10M
1,10.3666
2,10.5335
3,10.7409
4,9.60447
5,10.5326
6,10.3271
7,9.54054
8,10.251
9,9.80124
10,9.94717


In [5]:
# generate the mock galaxies
nside = 3  # number of subvolumes per side of cube to compute jackknife errors later
mockgals = get_mock(paramdicts[sample_num], m_th_arr[sample_num], nside)

1832182×5 Array{Float64,2}:
 1.62049e9   23.8908   178.352   69.3618  1.0
 1.62049e9   54.2385   147.212   86.2612  1.0
 1.62057e9   17.5035   142.147  121.991   1.0
 1.62057e9   30.9299   185.085  119.293   1.0
 1.62057e9    3.68962  147.032  121.787   1.0
 1.61988e9   46.9333   134.869   85.5845  1.0
 1.62057e9   10.0025   186.166  128.661   1.0
 1.62111e9   39.2183   196.085   89.3759  1.0
 1.62049e9   38.9772   181.184   67.3124  1.0
 1.62049e9   28.9453   165.548   72.4649  1.0
 1.6205e9    55.9567   140.022   83.2813  1.0
 1.62057e9   62.4508   162.055  106.218   1.0
 1.62057e9   20.3627   143.978  122.714   1.0
 ⋮                                           
 1.63291e9  217.366    391.477  211.086   0.0
 1.63299e9  244.322    354.392  238.514   0.0
 1.63298e9  221.873    373.901  252.498   0.0
 1.63299e9  258.998    358.065  259.844   0.0
 1.63246e9  253.761    351.039  264.374   0.0
 1.63236e9  206.285    343.378  235.826   0.0
 1.63238e9  249.434    337.411  199.124   0.0
 1.632

In [6]:
# compute the environment densities of the galaxies in the mock
deltas = get_rho(mockgals[:, 2:4], 400.0, 10.0)

1832182-element Array{Float64,1}:
  2.47743  
  2.51079  
  0.209179 
  0.20084  
  0.492711 
  1.52677  
  0.384302 
  1.06811  
  0.184162 
  0.834617 
  2.20224  
  0.392641 
  0.267554 
  ⋮        
  5.34611  
 -0.524667 
  3.86174  
  2.4274   
  1.63518  
  6.33013  
 -0.39958  
  0.0674136
  2.36068  
  1.39334  
  2.16054  
  2.01044  

In [7]:
# grab the halos to get stellar masses
halodf = load_halodat()

,halo_id,halo_upid,x,y,z,vx,vy,vz,hfoo1,hfoo2,halo_mvir,halo_mz_05,halo_mz_1,halo_mz_2,halo_mz_4,halo_mz_8,halo_ac,halo_ac_mpeak,halo_nfw_conc,halo_spin,hfoo3,halo_rvir,logMh,Mpeak,logMpeak,Mbar
1,1620490488,-1,23.8908,178.352,71.8972,254.19,-195.17,-253.54,758.46,780.72,1.303e14,1.117e14,1.016e14,9.289e13,6.113e13,4.885e13,0.51462,0.51462,4.34886,0.03889,1620490488,1.25141,14.1149,1.303e14,14.1149,2.03268e13
2,1620489517,-1,54.2385,147.212,83.8446,132.18,-243.42,241.66,746.88,787.27,1.031e14,1.009e14,9.834e13,8.531e13,6.773e13,5.168e13,0.602988,0.602988,6.31183,0.03163,1620489517,1.15745,14.0133,1.031e14,14.0133,1.60836e13
3,1620568787,-1,17.5035,142.147,121.233,206.25,52.6,75.76,800.83,833.51,1.023e14,9.305e13,8.565e13,8.209e13,7.753e13,5.354e13,0.465599,0.465599,8.92032,0.01292,1620568787,1.15445,14.0099,1.023e14,14.0099,1.59588e13
4,1620569234,-1,30.9299,185.085,116.149,100.4,211.18,314.45,759.85,808.61,8.734e13,8.65e13,8.481e13,7.882e13,6.102e13,4.87e13,0.474902,0.474902,8.7816,0.02291,1620569234,1.09519,13.9412,8.734e13,13.9412,1.3625e13
5,1620569405,-1,3.68962,147.032,119.642,318.78,-36.68,214.57,655.71,762.05,7.256e13,7.268e13,7.435e13,8.4e13,7.07e13,2.074e13,0.579871,0.608062,5.65232,0.0223,1620569405,1.02956,13.8607,8.4e13,13.9243,1.3104e13
6,1619882298,-1,46.9333,134.869,85.2907,-32.87,0.42,29.38,664.42,737.05,7.239e13,6.482e13,6.325e13,6.602e13,6.553e13,2.278e13,0.734503,0.734503,6.13975,0.06525,1619882298,1.02875,13.8597,7.239e13,13.8597,1.12928e13
7,1620570495,-1,10.0025,186.166,125.959,173.74,249.63,270.22,650.17,679.69,6.975e13,6.799e13,6.33e13,6.143e13,5.133e13,3.735e13,0.529195,0.529195,6.26543,0.04754,1620570495,1.01609,13.8435,6.975e13,13.8435,1.0881e13
8,1621113337,-1,39.2183,196.085,87.0058,-92.91,99.81,237.01,560.58,643.33,5.08e13,5.55e13,5.615e13,6.024e13,3.472e13,1.32e13,0.503591,0.516679,4.35401,0.08217,1621113337,0.914195,13.7059,6.024e13,13.7799,9.39744e12
9,1620494893,-1,38.9772,181.184,67.201,-119.43,12.79,11.14,568.63,639.93,4.591e13,4.567e13,4.37e13,4.085e13,3.21e13,1.713e13,0.473207,0.475291,6.38986,0.03874,1620494893,0.883866,13.6619,4.591e13,13.6619,7.16196e12
10,1620494673,-1,28.9453,165.548,73.521,28.3,-4.23,-105.61,586.93,628.04,4.428e13,4.125e13,3.994e13,3.841e13,3.449e13,2.469e13,0.381623,0.381623,7.59335,0.03238,1620494673,0.87328,13.6462,4.428e13,13.6462,6.90768e12


In [8]:
# make a merged dataframe of galaxies and halos
mockdf = DataFrame(delta = deltas, halo_id = mockgals[:, 1], subvol = mockgals[:, 6])

# keep only the central galaxies
mockdf = mockdf[mockgals[:, 5] .== 1, :]

# merge with the halos on halo_id
mockdf = join(mockdf, halodf, on = :halo_id)

,delta,halo_id,halo_upid,x,y,z,vx,vy,vz,hfoo1,hfoo2,halo_mvir,halo_mz_05,halo_mz_1,halo_mz_2,halo_mz_4,halo_mz_8,halo_ac,halo_ac_mpeak,halo_nfw_conc,halo_spin,hfoo3,halo_rvir,logMh,Mpeak,logMpeak,Mbar
1,2.47743,1.62049e9,-1,23.8908,178.352,71.8972,254.19,-195.17,-253.54,758.46,780.72,1.303e14,1.117e14,1.016e14,9.289e13,6.113e13,4.885e13,0.51462,0.51462,4.34886,0.03889,1620490488,1.25141,14.1149,1.303e14,14.1149,2.03268e13
2,2.51079,1.62049e9,-1,54.2385,147.212,83.8446,132.18,-243.42,241.66,746.88,787.27,1.031e14,1.009e14,9.834e13,8.531e13,6.773e13,5.168e13,0.602988,0.602988,6.31183,0.03163,1620489517,1.15745,14.0133,1.031e14,14.0133,1.60836e13
3,0.209179,1.62057e9,-1,17.5035,142.147,121.233,206.25,52.6,75.76,800.83,833.51,1.023e14,9.305e13,8.565e13,8.209e13,7.753e13,5.354e13,0.465599,0.465599,8.92032,0.01292,1620568787,1.15445,14.0099,1.023e14,14.0099,1.59588e13
4,0.20084,1.62057e9,-1,30.9299,185.085,116.149,100.4,211.18,314.45,759.85,808.61,8.734e13,8.65e13,8.481e13,7.882e13,6.102e13,4.87e13,0.474902,0.474902,8.7816,0.02291,1620569234,1.09519,13.9412,8.734e13,13.9412,1.3625e13
5,0.492711,1.62057e9,-1,3.68962,147.032,119.642,318.78,-36.68,214.57,655.71,762.05,7.256e13,7.268e13,7.435e13,8.4e13,7.07e13,2.074e13,0.579871,0.608062,5.65232,0.0223,1620569405,1.02956,13.8607,8.4e13,13.9243,1.3104e13
6,1.52677,1.61988e9,-1,46.9333,134.869,85.2907,-32.87,0.42,29.38,664.42,737.05,7.239e13,6.482e13,6.325e13,6.602e13,6.553e13,2.278e13,0.734503,0.734503,6.13975,0.06525,1619882298,1.02875,13.8597,7.239e13,13.8597,1.12928e13
7,0.384302,1.62057e9,-1,10.0025,186.166,125.959,173.74,249.63,270.22,650.17,679.69,6.975e13,6.799e13,6.33e13,6.143e13,5.133e13,3.735e13,0.529195,0.529195,6.26543,0.04754,1620570495,1.01609,13.8435,6.975e13,13.8435,1.0881e13
8,1.06811,1.62111e9,-1,39.2183,196.085,87.0058,-92.91,99.81,237.01,560.58,643.33,5.08e13,5.55e13,5.615e13,6.024e13,3.472e13,1.32e13,0.503591,0.516679,4.35401,0.08217,1621113337,0.914195,13.7059,6.024e13,13.7799,9.39744e12
9,0.184162,1.62049e9,-1,38.9772,181.184,67.201,-119.43,12.79,11.14,568.63,639.93,4.591e13,4.567e13,4.37e13,4.085e13,3.21e13,1.713e13,0.473207,0.475291,6.38986,0.03874,1620494893,0.883866,13.6619,4.591e13,13.6619,7.16196e12
10,0.834617,1.62049e9,-1,28.9453,165.548,73.521,28.3,-4.23,-105.61,586.93,628.04,4.428e13,4.125e13,3.994e13,3.841e13,3.449e13,2.469e13,0.381623,0.381623,7.59335,0.03238,1620494673,0.87328,13.6462,4.428e13,13.6462,6.90768e12


In [9]:
# now to add galaxy stellar masses according to the best fit stellar mass halo mass relation
ths = [18, 19, 20]

# read in the parameters
th = ths[sample_num]
params = squeeze(readdlm("../dat/optim/M$(@sprintf("%d", th))_mass_pars.dat"), 2)

# add stellar mass to dataframe
mockdf[:stelM] = gen_mstars(Array(mockdf[:Mpeak]), Array(mockdf[:Mbar]), params, fix_sig = false)

# and add log stellar mass
mockdf[:log10M] = log10.(mockdf[:stelM])

1092171-element Array{Float64,1}:
 10.7137 
 10.5356 
 10.4582 
 10.5707 
 10.5787 
 10.5075 
 10.7739 
 10.8607 
 10.6994 
 10.6661 
 10.6098 
 10.3536 
 10.5007 
  ⋮      
  8.78117
  8.79027
  9.08589
  8.68751
  9.28203
  8.94705
  8.62243
  9.25397
  8.81968
  8.68443
  8.61701
  9.00021

In [10]:
# now to get the mock numbers to match the quenched fraction in the data
nqbins = 20  # number of mass bins to use for the quenched fraction
massmins = [9.4, 9.8, 10.3]

# keep only the centrals within the massrange of interest for the modeling
mockdf = mockdf[massmins[sample_num] .<= mockdf[:log10M] .< 11.0, :]
    
# get the massbins for computing quenched fraction
quenmasses = linspace(massmins[sample_num], 11.0, nqbins + 1)

# compute fractions for the data
qfracs = zeros(nqbins)
for j in 1:nqbins
    ntot = size(datdf[quenmasses[j] .<= datdf[:log10M] .< quenmasses[j + 1], :])[1]
    nquens = size(quendf[quenmasses[j] .<= quendf[:log10M] .< quenmasses[j + 1], :])[1]
    qfracs[j] = nquens / ntot
end
println(qfracs)
    
mockdf = get_quenfrac(mockdf, quenmasses, qfracs)

[0.110749, 0.147558, 0.157205, 0.239595, 0.293044, 0.331203, 0.394505, 0.466238, 0.594438, 0.642308, 0.670851, 0.788976, 0.830986, 0.876543, 0.897833, 0.936255, 0.950311, 1.0, 0.983333, 1.0]

,delta,halo_id,halo_upid,x,y,z,vx,vy,vz,hfoo1,hfoo2,halo_mvir,halo_mz_05,halo_mz_1,halo_mz_2,halo_mz_4,halo_mz_8,halo_ac,halo_ac_mpeak,halo_nfw_conc,halo_spin,hfoo3,halo_rvir,logMh,Mpeak,logMpeak,Mbar,stelM,log10M
1,-0.449615,1.61861e9,-1,1.84362,10.4189,55.185,-342.17,-23.06,-240.63,202.61,227.3,1.874e12,1.867e12,1.879e12,1.796e12,1.231e12,9.013e11,0.47484,0.479527,6.89682,0.05261,1618609210,0.304327,12.2728,1.879e12,12.2739,2.93124e11,2.57553e9,9.41087
2,-0.624737,1.63092e9,-1,204.002,234.466,103.23,-206.81,-264.03,-82.28,159.56,174.18,9.664e11,6.949e11,6.653e11,6.435e11,6.092e11,5.465e11,0.558688,0.558688,5.59839,0.04884,1630921189,0.244045,11.9852,9.664e11,11.9852,1.50758e11,2.78437e9,9.44473
3,-0.166083,1.62097e9,-1,2.01861,150.445,375.889,150.99,-457.48,-0.19,102.1,116.67,1.987e11,1.875e11,1.697e11,1.555e11,1.579e11,1.195e11,0.304486,0.304486,11.9319,0.11176,1620972357,0.14404,11.2982,1.987e11,11.2982,3.09972e10,2.7623e9,9.44127
4,-0.649755,1.62619e9,-1,128.428,234.462,180.198,-265.48,-103.46,528.21,96.68,115.43,2.466e11,2.472e11,2.561e11,2.786e11,2.732e11,1.295e11,0.477509,0.558257,5.4457,0.06799,1626191617,0.154792,11.392,2.786e11,11.445,4.34616e10,2.63852e9,9.42136
5,1.83532,1.62597e9,-1,170.705,237.079,2.87335,68.82,77.69,256.31,123.38,134.02,2.916e11,2.916e11,2.88e11,2.797e11,2.52e11,2.218e11,0.234851,0.234851,16.3317,0.08206,1625967300,0.163687,11.4648,2.916e11,11.4648,4.54896e10,2.63492e9,9.42077
6,2.7693,1.62036e9,-1,24.6398,132.684,359.48,277.39,254.59,23.1,120.41,125.66,2.608e11,2.567e11,2.448e11,2.283e11,1.969e11,1.68e11,0.369976,0.369976,13.718,0.05617,1620359066,0.157708,11.4163,2.608e11,11.4163,4.06848e10,2.54484e9,9.40566
7,-0.441276,1.62444e9,-1,171.35,94.9889,246.349,-436.15,-187.1,281.35,115.11,126.6,3.123e11,3.111e11,3.046e11,3.247e11,2.886e11,1.904e11,0.392427,0.418653,11.7278,0.0693,1624442522,0.167472,11.4946,3.247e11,11.5115,5.06532e10,2.93457e9,9.46755
8,0.0257177,1.63231e9,-1,257.206,315.865,152.035,-396.55,-76.55,-121.1,126.19,141.84,3.519e11,3.59e11,3.815e11,3.738e11,2.372e11,2.188e11,0.30499,0.318027,13.4816,0.04343,1632311283,0.174271,11.5464,3.815e11,11.5815,5.9514e10,2.69894e9,9.43119
9,0.951365,1.62453e9,-1,156.752,73.4321,291.893,93.36,-276.44,34.54,111.95,130.4,2.111e11,2.099e11,2.088e11,2.094e11,1.898e11,1.644e11,0.249645,0.262452,15.6769,0.0526,1624525355,0.146976,11.3245,2.111e11,11.3245,3.29316e10,2.55716e9,9.40776
10,0.259214,1.63772e9,-1,370.215,360.958,83.2546,-358.78,152.2,-392.59,122.29,144.3,4.205e11,3.998e11,3.838e11,3.442e11,3.507e11,2.372e11,0.387768,0.387768,7.11511,0.0682,1637722374,0.184929,11.6238,4.205e11,11.6238,6.5598e10,2.78372e9,9.44463


In [11]:
# finally leave only the relevant columns in the mock for modeling
mockdf = mockdf[:, [:halo_id, :halo_mvir, :halo_rvir, :halo_nfw_conc, :halo_spin, :halo_mz_05, :halo_mz_1,
                    :delta, :stelM, :log10M]]
outfile = "../dat/mocks/M$(@sprintf("%d", th))_cenquen_mock.csv"
CSV.write(outfile, mockdf)

CSV.Sink{Void,DataType}(    CSV.Options:
        delim: ','
        quotechar: '"'
        escapechar: '\\'
        missingstring: ""
        dateformat: nothing
        decimal: '.'
        truestring: 'true'
        falsestring: 'false'
        internstrings: true, IOBuffer(data=UInt8[...], readable=true, writable=true, seekable=true, append=false, size=0, maxsize=Inf, ptr=1, mark=-1), "../dat/mocks/M18_cenquen_mock.csv", 92, true, String["halo_id", "halo_mvir", "halo_rvir", "halo_nfw_conc", "halo_spin", "halo_mz_05", "halo_mz_1", "delta", "stelM", "log10M"], 10, false, Val{false})